# Better than transkribus

In [1]:
#!pip install tabula-py
#!pip install JPype1
#!pip install xlsxwriter
#!pip install pymupdf
#!pip install pandas openpyxl

In [145]:
import tabula
import pandas as pd

# Path to the PDF file
pdf_path = "C:/Users/Danny/sciebo/Institut/Rand_McNally_1909/rmbd_190901_04_kansas.pdf"

# Extract tables from the PDF
#area = [50, 20, 500, 750]
tables = tabula.read_pdf(pdf_path, pages='all', columns=[34.5, 70.4, 120.0, 150.5, 200.0, 266.9, 300.5, 328.6, 400.0, 450.2, 480.5, 550.0])



# Export the extracted tables to an Excel file
excel_path = "C:/Users/Danny/sciebo/Institut/Rand_McNally_1909/extracted_tables_kansas.xlsx"

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    for i, table in enumerate(tables):
        table.to_excel(writer, sheet_name=f'Table_{i + 1}', index=False) 

In [46]:
import pandas as pd
from openpyxl import load_workbook

def split_columns_in_excel_by_index_replace(file_path, sheet_name, columns_to_split_indices, delimiter):
    # Load the workbook and the specific sheet
    book = load_workbook(file_path)
    
    if sheet_name not in book.sheetnames:
        print(f"Sheet '{sheet_name}' not found.")
        return

    # Load the DataFrame from the specified sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Split the specified columns and replace them in the original DataFrame
    for col_index in sorted(columns_to_split_indices, reverse=True):  # Sort indices in reverse order
        if col_index < df.shape[1]:  # Ensure the index is within range
            column_name = df.columns[col_index]
            # Split the column based on the provided delimiter, expand to new columns
            split_columns = df[column_name].str.split(delimiter, expand=True, n=1)

            # Create new column names
            num_splits = split_columns.shape[1]
            new_column_names = [f"{column_name}_part_{i + 1}" for i in range(num_splits)]

            # Drop the original column from the DataFrame
            df.drop(columns=[column_name], inplace=True)

            # Insert the new columns at the position of the original column
            for i, new_col_name in enumerate(new_column_names):
                df.insert(col_index + i, new_col_name, split_columns[i])

    # Use openpyxl's workbook to get the active writer and modify only the specific sheet
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        # Save back only the modified sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Modified sheet '{sheet_name}' saved back into {file_path}")



In [47]:
import pandas as pd
split_columns_in_excel_by_index_replace(r"C:\Users\Danny\sciebo\Institut\Rand_McNally_1909\extracted_tables_kansas.xlsx", "Table_1", [11], " ")

Modified sheet 'Table_1' saved back into C:\Users\Danny\sciebo\Institut\Rand_McNally_1909\extracted_tables_kansas.xlsx


In [7]:
import pandas as pd
from openpyxl import load_workbook

def replace_patterns_in_all_columns(file_path, replacements):
    """
    Replace specific patterns in all columns across all sheets in an Excel file, while avoiding changing empty cells (NaN).
    
    Parameters:
    - file_path: Path to the Excel file.
    - replacements: A dictionary with patterns as keys and their replacements as values.
    """
    # Load the entire Excel workbook into a dictionary of DataFrames (one per sheet)
    excel_file = pd.read_excel(file_path, sheet_name=None)  # Load all sheets
    
    # Iterate through each sheet and apply the replacements
    for sheet_name, df in excel_file.items():
        
        # Iterate through all columns in the DataFrame
        for col in df.columns:
            # Only apply replacement if the cell is not empty (skip NaN values)
            df[col] = df[col].apply(lambda x: x if pd.isna(x) else replace_patterns(str(x), replacements))
        
        # Save the modified DataFrame back into the dictionary
        excel_file[sheet_name] = df

    # Save the modified DataFrames back to the Excel file
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        for sheet_name, df in excel_file.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        

    print(f"Replaced patterns in all sheets and saved back to '{file_path}'")


# Example usage:
file_path = r"C:\Users\Danny\sciebo\Institut\Rand_McNally_1909\extracted_tables_kansas.xlsx"
replacements = {
    'CO0': '000',  
    'coo': '000',  
    'COO': '000',  
    'C00': '000',  
    'C0O': '000',  
    '0C0': '000',  
    'ooo': '000',
    '0CO': '000',
    'OC0': '000',
    'TOO': '000',
}

replace_patterns_in_all_columns(file_path, replacements)



Replaced patterns in all sheets and saved back to 'C:\Users\Danny\sciebo\Institut\Rand_McNally_1909\extracted_tables_kansas.xlsx'


In [149]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

def delete_all_letters(file_path, column_indices, sheet_name):
    # Read the specific sheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Remove letters from specified columns using column indices
    for col in column_indices:
        if col < len(df.columns):  # Ensure the index is valid
            if df.iloc[:, col].dtype != 'object':
                raise ValueError(f"Column index {col} is not of type object. Chose wrong column!")
            
            # Print before modification
            print(f"Before replacement in column {col}:")
            print(df.iloc[:, col])

            # Remove letters and keep numbers, commas, and spaces
            df.iloc[:, col] = df.iloc[:, col].astype(str).str.replace(r'[^0-9, ]', '', regex=True)

            # Print after modification
            print(f"After replacement in column {col}:")
            print(df.iloc[:, col])
        else:
            print(f"Column index {col} is out of range. Skipping.")
    book=load_workbook(file_path)
   # Use openpyxl's workbook to get the active writer and modify only the specific sheet
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        # Save back only the modified sheet
        
        df.to_excel(writer, sheet_name=sheet_name, index=False)


    print(f"Replaced patterns in '{sheet_name}' and saved back to '{file_path}' with columns {column_indices}.")


In [151]:
import pandas as pd
file_path = r"C:/Users/Danny/sciebo/Institut/Rand_McNally_1909/extracted_tables_kansas.xlsx"
sheet_name = "Table_1"
range=[11]
delete_all_letters(file_path,range,sheet_name)

Before replacement in column 11:
0       Pb o p . I ITS.
1                   NaN
2     l  6,500 5 62,500
3                   NaN
4        25,000 381,290
5        10,000 100.000
6      29,000 i 347,000
7        18 500 225,000
8                   NaN
9          5,000 58,000
10               10,COO
11              103,500
12         1,600 57,000
13                  NaN
14         8,000 90,000
15         1,480 63,660
16                  NaN
17         5,000 71,000
18        10,240 90,590
19                  NaN
20        2,000 175,000
21                  NaN
22        6,900 211,000
23        3,000 150,000
24                  NaN
25               25,000
26         7,000 80,000
27               80,000
28                  NaN
29        7,500 106,000
30                  NaN
31           500 60,000
32         7,000 83,500
33         1,600 40,000
34         3,500 80,000
35         1,100 80,000
36       14,370 112,720
37         1,010 66,840
38         1,000 55,000
39                  NaN
40     